In [ ]:
# Figure 5

In [ ]:
# Modules

In [ ]:
# final modules list
# established signal
exhaustion.markers <- list(c("PDCD1", "LAG3", "TIGIT", "HAVCR2"))
sce <- AddModuleScore(object = sce, 
                      features = exhaustion.markers, 
                      name = "exhaustion.markers")

In [ ]:
# lung cNMF modules
file_path <- '/sc/arion/work/kimg20/aging/supp_table_5.csv'
gene_table <- read.csv(file_path)


module_names <- gene_table[, 2]  
gene_table_no_celltype <- gene_table[, -c(1, 2)]
transposed_gene_table <- t(gene_table_no_celltype)

markers_list <- lapply(1:ncol(transposed_gene_table), function(i) {
  col <- transposed_gene_table[, i]
  col <- col[!is.na(col)]  # remove NA values
  as.character(col)        # convert to character vector
})

names(markers_list) <- as.character(module_names)

print(names(markers_list))

for (name in names(markers_list)) {
  markers <- markers_list[[name]]
  print(markers)
  sce <- AddModuleScore(object = sce, features = list(markers), name = name)
}

### Figure 5A

### Figure 5B - Lollipop Proportion

In [ ]:
library(ggplot2)
library(dplyr)
library(stats)
library(scales)

composition_analysis_lollipop <- function(compartment_seurat_object, category_meta, category_idents, out_dir, refined_label_meta, sampleID_meta, name) {
  
  obj <- data
  obs <- obj@meta.data
  
  obs_filtered <- obs %>% filter(!!as.name(category_meta) %in% c("Young", "Old"))
  
  Dout <- table(obs_filtered[[refined_label_meta]], obs_filtered[[sampleID_meta]])
  Dout <- prop.table(Dout, margin = 2)  # normalize by columns
  subtypes <- rownames(Dout)            # nell types
  
  age_groups <- obs_filtered %>% select(!!as.name(category_meta), !!as.name(sampleID_meta)) %>% distinct()
  
  #  results data frame
  correlation_results <- data.frame(
    CellType = subtypes,
    Spearman_Rho = numeric(length(subtypes)),
    P.Value = numeric(length(subtypes)),
    Direction = character(length(subtypes))
  )
  
  for (i in seq_along(subtypes)) {
    subtype <- subtypes[i]
    proportions <- Dout[subtype, ] # proportions
    age_values <- age_groups[[category_meta]][match(names(proportions), age_groups[[sampleID_meta]])] # map the corresponding age group for each sample (Tube_id)
    
    age_binary <- ifelse(age_values == "Old", 1, 0)
    spearman_result <- cor.test(proportions, age_binary, method = "spearman") # Spearman correlation between proportions and age
    
    # results
    correlation_results$Spearman_Rho[i] <- spearman_result$estimate
    correlation_results$P.Value[i] <- spearman_result$p.value
    
    # medians of Young and Old to determine direction
    median_young <- median(proportions[age_values == "Young"])
    median_old <- median(proportions[age_values == "Old"])
    
    if (median_young > median_old) {
      correlation_results$Direction[i] <- "Young > Old"
    } else if (median_young < median_old) {
      correlation_results$Direction[i] <- "Young < Old"
    } else {
      correlation_results$Direction[i] <- "No Difference"
    }
  }
  
  correlation_results$FDR <- p.adjust(correlation_results$P.Value, method = "fdr")
  correlation_results$log_fdr_corrected_p_value <- -log10(correlation_results$FDR)

  correlation_results <- correlation_results %>%
      mutate(
        significant = FDR < 0.05,
        log_fdr_corrected_p_value = ifelse(Direction == "Young < Old", -log10(FDR), log10(FDR)),
        color = ifelse(significant,
                       ifelse(Direction == "Young < Old", "#D8423D", "#0091CA"),  # Blue for Young < Old, Red for Young > Old
                       "#D3D3D3") 
      )

    
  positive_threshold <- -log10(0.05) 
  negative_threshold <- log10(0.05) 
  
  lollipop_plot <- ggplot(correlation_results, aes(x = reorder(CellType, log_fdr_corrected_p_value), y = log_fdr_corrected_p_value)) +
    geom_segment(aes(xend = CellType, y = 0, yend = log_fdr_corrected_p_value), color = "gray") +
    geom_point(aes(color = color),  size = 1) + 
    scale_color_identity() +
    theme_minimal() +
    coord_flip() + 
    labs(
      title = "Spearman Correlation of Cell Type Proportions (Young vs Old)",
      x = "Cell Type",
      y = "-log10(p-val)",
      color = "Direction",
      size = "Spearman Rho"
    ) +
    theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
    theme_minimal() +
      theme(
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 0.5, size = 6),
        axis.ticks = element_line(color = "black"),  
        axis.ticks.length = unit(0.1, "cm"), 
        text = element_text(size = 6),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        axis.line.y.left = element_line(size = 0.5),
        axis.line = element_line(size = 0.5),
        legend.position = "none"
          )+
        
        geom_hline(yintercept = positive_threshold, linetype = "dashed", color = "gray") +
        geom_hline(yintercept = negative_threshold, linetype = "dashed", color = "gray") +
        geom_hline(yintercept = log10(1),  color = "black") 

  
  plot_file <- paste0(out_dir, "/", name, "_spearman_lollipop.pdf")
  ggsave(plot_file, plot = lollipop_plot, height = 1.5, width = 2, dpi = 300)
  
  print(paste("Plot saved to", plot_file))
  
  return(correlation_results) 
}


composition_analysis_lollipop(
  compartment_seurat_object = sce,
  category_meta = 'Age_group3',         
  category_idents = c("Young", "Old"),
  out_dir = "/sc/arion/work/kimg20/aging/composition_analysis/",
  refined_label_meta = "Cluster_names", 
  sampleID_meta = "Tube_id",
  name = "total_FDR"
)


In [ ]:
# lung
file_path <- "/sc/arion/work/kimg20/aging/HLCA_CD8T_Label_Transfer_Spearman.csv"
df <- read.csv(file_path)
out_dir = "/sc/arion/work/kimg20/aging/composition_analysis/"
print(df)

df$color <- ifelse(df$Significance == "Not Significant", "#D3D3D3",  
              ifelse(df$gradients > 0, "#0091CA", 
              ifelse(df$gradients < 0, "#D8423D", "#D3D3D3"))) 
df <- df %>% arrange(directed_log_pval)

p <-  ggplot(df, aes(y = reorder(names, directed_log_pval), x = directed_log_pval)) +
      geom_segment(aes(y = names, yend = names, x = 0, xend = directed_log_pval), color = "gray") +
      geom_point(aes(color = color), size = 1) +  
      scale_color_identity() +  
    
      geom_vline(xintercept = -log10(0.05), linetype = "dashed", color = "gray") +
      geom_vline(xintercept = log10(0.05), linetype = "dashed", color = "gray") +
      geom_vline(xintercept = log10(1),  color = "black") +
    
      labs(
        title = "Lollipop Plot of Directed Log P-values by Cell Type",
        x = "Directed log10(p-value)",
        y = "Cell Types"
      ) +
        theme_minimal() +
          theme(
            axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 0.5, size = 6),
            axis.ticks = element_line(color = "black"),  
            axis.ticks.length = unit(0.1, "cm"), 
            text = element_text(size = 6),
            panel.grid.major = element_blank(),
            panel.grid.minor = element_blank(),
            panel.border = element_blank(),
            axis.line.y.left = element_line(size = 0.5),
            axis.line = element_line(size = 0.5),
            legend.position = "none"
              ) 

plot_file <- paste0(out_dir, "/", name, "Fig5B_lung_lollipop.pdf")
ggsave(plot_file, plot = p, height = 1.5, width = 2, dpi = 300)

### Figure 5C - Label Transfer

### Figure 5D

In [ ]:
# Spearman Blood!!!!!!!!!!!!!!!!!!!!!
file <- "lollipops1014/cd8/"
metadata <- sce@meta.data


features <- c("Tm21", "Tm41", "Tm71", "Tm81", "Tm121", "exhaustion.markers1")

by_sample <- metadata %>%
  group_by(Tube_id, Age) %>%
  summarise(across(all_of(features), ~mean(.x, na.rm = TRUE)), .groups = 'drop')

# Caverage IFNG expression
avg_expr <- AverageExpression(object = sce, features = "IFNG", assay = "RNA", group.by = c('Tube_id', 'Age'))$RNA
avg_expr <- as.data.frame(as.matrix(avg_expr))

avg_expr_long <- avg_expr %>%
  pivot_longer(cols = everything(), 
               names_to = "Tube_id_Age", 
               values_to = "IFNG") %>%
  separate(Tube_id_Age, into = c("Tube_id", "Age"), sep = "_")


by_sample <- by_sample %>%
  mutate(Age = as.numeric(Age))

avg_expr_long <- avg_expr_long %>%
  mutate(Age = as.numeric(Age))

by_sample <- by_sample %>%
  left_join(avg_expr_long, by = c("Tube_id", "Age"))


p_values <- data.frame(feature = features, p_value = NA, rho = NA)

for (feature in features) {
  test_result <- cor.test(by_sample[[feature]], by_sample$Age, method = "spearman", use = "complete.obs")
  
  p_values$p_value[p_values$feature == feature] <- test_result$p.value
  p_values$rho[p_values$feature == feature] <- test_result$estimate
}

p_values <- p_values %>%
  mutate(fdr_corrected_p_value = p.adjust(p_value, method = "fdr"),
         log_p_value = -log10(p_value),
         log_fdr_corrected_p_value = ifelse(rho > 0, 
                                            -log10(fdr_corrected_p_value), 
                                            log10(fdr_corrected_p_value)))

positive_threshold <- -log10(0.05) 
negative_threshold <- log10(0.05) 

p_values <- p_values %>%
  mutate(significant = log_fdr_corrected_p_value >= positive_threshold | log_fdr_corrected_p_value <= negative_threshold)
p_values <- p_values %>%
  mutate(color = ifelse(significant, 
                        ifelse(log_fdr_corrected_p_value > 0, "#D8423D", "#0091CA"), 
                        "#D3D3D3"))


plot <- ggplot(p_values, aes(x = reorder(feature, -log_fdr_corrected_p_value),  
                             y = log_fdr_corrected_p_value)) +
  geom_segment(aes(x = reorder(feature, -log_fdr_corrected_p_value), 
                   xend = reorder(feature, -log_fdr_corrected_p_value), 
                   y = 0, yend = log_fdr_corrected_p_value),
               color = "#D3D3D3") +
  geom_point(aes(color = color), size = 1) +
  scale_color_identity() +  
  geom_hline(yintercept = -log10(0.05), linetype = "dashed", color = "#D3D3D3") +
  geom_hline(yintercept = log10(0.05), linetype = "dashed", color = "#D3D3D3") +
  geom_hline(yintercept = log10(1),  color = "black") +

  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 0.5, size = 6),
    axis.ticks = element_line(color = "black"),  
    axis.ticks.length = unit(0.1, "cm"), 
    text = element_text(size = 6),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    panel.border = element_blank(),
    axis.line.y.left = element_line(size = 0.5),
    axis.line = element_line(size = 0.5)
  ) +
  labs(x = "Features", y = "-log10(p)") 


ggsave(filename = paste0(file, "blood_cd8_spearmann_mean.pdf"), plot = plot, width = 1, height = 1.5)


In [ ]:
# Spearman Lung!!!!!!!!!!!!!!!!!!!!!
file <- "lollipops1014/cd8/"

metadata <- sce@meta.data

features <- c("Tm21", "Tm41", "Tm71", "Tm81", "Tm121", "exhaustion.markers1")

by_sample <- metadata %>%
  group_by(sample, Age) %>%
  summarise(across(all_of(features), ~mean(.x, na.rm = TRUE)), .groups = 'drop')

avg_expr <- AverageExpression(object = sce, features = "IFNG", assay = "RNA", group.by = c('sample', 'Age'))$RNA
avg_expr <- as.data.frame(as.matrix(avg_expr))

avg_expr_long <- avg_expr %>%
  pivot_longer(cols = everything(), 
               names_to = "sample_Age", 
               values_to = "IFNG") %>%
  separate(sample_Age, into = c("sample", "Age"), sep = "_")


by_sample <- by_sample %>%
  mutate(Age = as.numeric(Age))

avg_expr_long <- avg_expr_long %>%
  mutate(Age = as.numeric(Age))

by_sample <- by_sample %>%
  left_join(avg_expr_long, by = c("sample", "Age"))



p_values <- data.frame(feature = features, p_value = NA, rho = NA)
for (feature in features) {
  test_result <- cor.test(by_sample[[feature]], by_sample$Age, method = "spearman", use = "complete.obs")
  
  p_values$p_value[p_values$feature == feature] <- test_result$p.value
  p_values$rho[p_values$feature == feature] <- test_result$estimate
}

p_values <- p_values %>%
  mutate(fdr_corrected_p_value = p.adjust(p_value, method = "fdr"),
         log_p_value = -log10(p_value),
         log_fdr_corrected_p_value = ifelse(rho > 0, 
                                            -log10(fdr_corrected_p_value), 
                                            log10(fdr_corrected_p_value)))

positive_threshold <- -log10(0.05) 
negative_threshold <- log10(0.05)

p_values <- p_values %>%
  mutate(significant = log_fdr_corrected_p_value >= positive_threshold | log_fdr_corrected_p_value <= negative_threshold)

p_values <- p_values %>%
  mutate(color = ifelse(significant, 
                        ifelse(log_fdr_corrected_p_value > 0, "#D8423D", "#0091CA"), 
                        "#D3D3D3"))


plot <- ggplot(p_values, aes(x = reorder(feature, -log_fdr_corrected_p_value),  
                             y = log_fdr_corrected_p_value)) +
  geom_segment(aes(x = reorder(feature, -log_fdr_corrected_p_value),  
                   xend = reorder(feature, -log_fdr_corrected_p_value), 
                   y = 0, yend = log_fdr_corrected_p_value),
               color = "#D3D3D3") +
  geom_point(aes(color = color), size = 1) +
  scale_color_identity() +  
  geom_hline(yintercept = -log10(0.05), linetype = "dashed", color = "#D3D3D3") +
  geom_hline(yintercept = log10(0.05), linetype = "dashed", color = "#D3D3D3") +
  geom_hline(yintercept = log10(1),  color = "black") +

  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 0.5, size = 6),
    axis.ticks = element_line(color = "black"),  
    axis.ticks.length = unit(0.1, "cm"), 
    text = element_text(size = 6),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    panel.border = element_blank(),
    axis.line.y.left = element_line(size = 0.5),
    axis.line = element_line(size = 0.5)
  ) +
  labs(x = "Features", y = "-log10(p-value)") 


ggsave(filename = paste0(file, "lung_nk_spearmann_mean.pdf"), plot = plot, width = 1, height = 1.5)


### Figure 5E

In [ ]:
file_path <- "test/test_proportions.pdf"

metadata_df <- metadata_df %>%
  count(Age_group3, cloneSize) %>%
  group_by(Age_group3) %>%
  mutate(Total = sum(n), Proportion = n / Total) %>%
  ungroup()

metadata_df$cloneSize <- factor(metadata_df$cloneSize, levels = rev(levels(metadata_df$cloneSize)))

In [ ]:
plot1 <- ggplot(metadata_df, aes(x = Age_group3, y = Proportion, fill = cloneSize)) +
  geom_bar(stat = "identity", width = 0.5, position = "stack") +
  scale_y_continuous(labels = scales::percent_format(accuracy = 1)) +
  labs(x = "Age", y = "Proportion", fill = "Clone Size") +
  theme_minimal() +
  theme(
    text = element_text(size = 6, colour = "black"),
    axis.text.x = element_text(angle = 0, hjust = 0.5, colour = "black", margin = margin(t = -5)),
    axis.text.y = element_text(angle = 0, colour = "black", margin = margin(r = -5)),
    axis.title.x = element_text(margin = margin(t = 2, r = 0, b = 0, l = 0)),
    axis.title.y = element_text(margin = margin(t = 0, r = 2, b = 0, l = 6)),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    legend.position = "none",
    plot.title = element_text(margin = margin(t = 0, b = 0)),
    plot.caption = element_text(margin = margin(t = 0, b = 0)),
    plot.margin = margin(t = 0, r = 0, b = 0, l = 0, unit = "cm"),
    legend.key.size = unit(2, 'mm'),
    legend.spacing = unit(0.5, 'mm'),
    legend.margin = margin(t = 0, r = 0, b = 0, l = 0, unit = "cm")
  ) +
  scale_x_discrete(labels = c("<40", "40-59", expression("">=60)))
print(plot1)
# Save the plot to a PDF file
#ggsave(file_path, plot = plot1, device = "pdf", width = 1, height = 1)

### Figure 5F, Entropy Analysis

In [ ]:
plot1 <- clonalDiversity(sce, 
                    cloneCall = "gene",
                    group.by = "Tube_id",
                    exportTable = TRUE)

In [ ]:
# Adding age
metadata_df <- sce@meta.data
age_data <- metadata_df[, c("Tube_id", "Age")]
plot2 <- merge(plot1, age_data, by = "Tube_id", all.x = TRUE)
head(plot2)

In [ ]:
plot2 <- plot2 %>%
  arrange(Age) %>%
  mutate(Tube_id = factor(Tube_id, levels = unique(Tube_id)))

plot2 <- plot2[!duplicated(plot2$Tube_id), ]
head(plot2)

In [ ]:

plot <- ggplot(plot2, aes(x = Tube_id, y = norm.entropy)) +
  geom_point(position = position_jitter(width = 0.1, height = 0), size = 2, alpha = 0.7) +
  labs(title = "Dot Plot of norm.entropy by Tube_id", 
       x = "Tube ID", 
       y = "Normalized Entropy") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))


In [ ]:
plot1_filtered <- plot1[plot1$norm.entropy < 1, ]
ggplot(plot1_filtered, aes(x = Tube_id, y = norm.entropy)) +
  geom_point(position = position_jitter(width = 0.1, height = 0), size = 2, alpha = 0.7) +
  labs(title = "Dot Plot of norm.entropy by Tube_id", 
       x = "Tube ID", 
       y = "Normalized Entropy") +
  theme_minimal()

In [ ]:
## by age group instead of tube id, box plot
metadata_df <- sce@meta.data
age_data <- metadata_df[, c("Tube_id", "Age_group3")]
plot2 <- merge(plot1, age_data, by = "Tube_id", all.x = TRUE)

plot2 <- plot2 %>%
  arrange(Age_group3) %>%
  mutate(Tube_id = factor(Tube_id, levels = unique(Tube_id)))

plot2 <- plot2[!duplicated(plot2$Tube_id), ]
head(plot2)

In [ ]:
palette <- c("#0091CA", "#D3D3D3", "#D8423D")
my_comparisons <- list(
  c('Young', 'Intermediate'),
  c('Young', 'Old'),
  c('Intermediate', 'Old')
)


plot <- ggplot(plot2, aes(x = Age_group3, y = norm.entropy, color = Age_group3)) +
        geom_boxplot(size = 0.5, outlier.shape = NA, fatten = 1.5) +
        geom_jitter(width = 0.25, size = 0.3, stroke = 0.1) +
        scale_color_manual(values = palette) +
        theme_minimal() +
        theme(
                text = element_text(size = 6, colour = "black"),
                plot.title = element_text(size = 6, hjust = 0.5, margin = margin(t = -1.5)),
                axis.line = element_line(linewidth = 0.3, colour = "black"),
                axis.text.x = element_text(angle = 0, hjust = 0.5, colour = "black", margin = margin(t = -1.5)),
                axis.text.y = element_text(angle = 0, colour = "black", margin = margin(r = -1)),
                panel.grid.major = element_blank(),
                panel.grid.minor = element_blank(),
                legend.position = "none"
              ) +
         scale_x_discrete(labels = c("<40", "40-59", expression(">=60"))) +
         stat_compare_means(comparisons = my_comparisons, method = "wilcox.test", label = "p.signif", p.adjust = 'fdr', size = 2) # p.adj.format, p.signif

  
# Open a PDF device to save the plot
pdf("entropy_wpval.pdf", width = 1.1, height = 1.7)
print(plot)
dev.off()

In [ ]:
group1 <- plot2$norm.entropy[plot2$Age_group3 == 'Young']
group2 <- plot2$norm.entropy[plot2$Age_group3 == 'Intermediate']
group3 <- plot2$norm.entropy[plot2$Age_group3 == 'Old']

# Perform Wilcoxon tests for each pair and store raw p-values
p_val1 <- wilcox.test(group1, group2)$p.value
p_val2 <- wilcox.test(group1, group3)$p.value
p_val3 <- wilcox.test(group2, group3)$p.value

# Combine raw p-values into a vector
raw_pvals <- c(p_val1, p_val2, p_val3)

# Apply FDR correction
fdr_pvals <- p.adjust(raw_pvals, method = "fdr")

# Print raw and FDR-corrected p-values
comparison_names <- c("Young vs Intermediate", "Young vs Old", "Intermediate vs Old")
p_val_table <- data.frame(Comparison = comparison_names, Raw_P_Value = raw_pvals, FDR_P_Value = fdr_pvals)

print(p_val_table)

### Figure 5G

In [ ]:
#features <- c("exhaustion.markers1", "senmayo.markers1", "jasonji.markers1", "ifng.markers1", "stress.markers1", "atp.markers1", "cytotoxic.markers1", "chemokine.markers1")
#features <- c("X11", "X21", "X31", "X41", "X51", "X61", "X71", "X81", "X91", "X101", "X111", "X121", "X131", "X141", "X151", "X161", "X171", "X181", "X191", "X201")
#features <- c("X21", "X41", "X51", "exhaustion.markers1", "ifng.markers1")
# features <- c("exhaustion.markers1", "senmayo.markers1", "jasonji.markers1", "ifng.markers1", "stress.markers1", "atp.markers1", "cytotoxic.markers1", "chemokine.markers1", "sen.markers1")
# features <- c("X41", "X51")

# ======================================================
file <- "clonesize/blood_cd8/mean_star/"
metadata <- sce@meta.data
features <- c("Tm21", "Tm41", "Tm71", "Tm81", "Tm121", "exhaustion.markers1")

by_sample <- metadata %>%
  group_by(Tube_id, Age_group3, cloneSize) %>%
  summarise(across(all_of(features), ~mean(.x, na.rm = TRUE)), .groups = 'drop')


avg_expr <- AverageExpression(object = sce, features = "IFNG", assay = "RNA", group.by = c('Tube_id', 'Age_group3', 'cloneSize'))$RNA
avg_expr <- as.data.frame(as.matrix(avg_expr))


avg_expr_long <- avg_expr %>%
  pivot_longer(cols = everything(), 
               names_to = "Tube_id_Age_group3_cloneSize", 
               values_to = "IFNG") %>%
  separate(Tube_id_Age_group3_cloneSize, into = c("Tube_id", "Age_group3", "cloneSize"), sep = "_")

by_sample <- by_sample %>%
  left_join(avg_expr_long, by = c("Tube_id", "Age_group3", "cloneSize"))

#features <- c("Tm21", "Tm41", "Tm71", "Tm81", "Tm121", "IFNG", "exhaustion.markers1")
features <- c("IFNG")

# ======

dir.create(file, recursive = TRUE, showWarnings = FALSE)

my_comparisons <- list(
    c('Small (1 < X <= 5)', 'Single (0 < X <= 1)'),
    c('Large (5 < X <= 500)', 'Small (1 < X <= 5)'),
    c('Large (5 < X <= 500)', 'Single (0 < X <= 1)')
)
palette = c("#929292", "#616161", "#303030")

for (feature in features) {
  split_feature <- unlist(strsplit(feature, ".", fixed = TRUE))

  short_title <- split_feature[1]
  short_title <- toTitleCase(short_title)
  file_name <- paste0(short_title, ".pdf")
  complete_file_path <- paste0(file, file_name)
  file_name <- paste0(short_title, ".pdf")
  complete_file_path <- paste0(file, file_name)


  if (!dir.exists(directory_path)) {
    dir.create(directory_path, recursive = TRUE, showWarnings = FALSE)
  }

  plot_data <- by_sample %>%
    select(Tube_id, Age_group3, cloneSize, all_of(feature)) %>%
    na.omit()
  
  if(nrow(plot_data) == 0) {
    print(paste("No data for feature:", feature))
    next
  }
  
  plot_data$cloneSize <- factor(plot_data$cloneSize, levels = c('Single (0 < X <= 1)', 'Small (1 < X <= 5)', 'Large (5 < X <= 500)'))
  #plot_data$Age_group3 <- factor(plot_data$Age_group3, levels = sort(unique(plot_data$Age_group3)))
  plot_data$Age_group3 <- factor(plot_data$Age_group3, levels = c("Young", "Intermediate", "Old"))
  overall_max <- max(plot_data[[feature]], na.rm = TRUE)

    stat.test <- plot_data %>%
        group_by(Age_group3) %>%
        wilcox_test(formula = as.formula(paste(feature, "~ cloneSize")), comparisons = my_comparisons) %>%
        adjust_pvalue(method = "fdr") %>%
    mutate(
      y.position = overall_max * (0.7 + (row_number() * 0.1)), 
      p.adj = signif(p.adj, 2) 
    )

  print(stat.test)
  plot1 <- ggboxplot(plot_data, x = "cloneSize", y = feature, color = "cloneSize", palette = palette, 
                     facet.by = "Age_group3", size = 0.5, short.panel.labs = TRUE, outlier.shape = NA, fatten = 1.5) +
           geom_jitter(aes(color = cloneSize), width = 0.25, size = 0.3, stroke = 0.1) + 
           theme_minimal() +
           theme(text = element_text(size = 6, colour = "black"),
                   plot.title = element_text(size = 6, hjust = 0.5, margin = margin(t = -1.5)),
                   axis.line = element_line(linewidth = 0.3, colour = "black"),
                   axis.text.x = element_text(angle = 0, hjust = 0.5, colour = "black", margin = margin(t = -1.5)),
                   axis.text.y = element_text(angle = 0, colour = "black", margin = margin(r = -1)),
                   panel.grid.major = element_blank(),
                   panel.grid.minor = element_blank(),
                   legend.position = "none") +
           labs(y = feature, x = "") +
           #xlab("cloneSize") + 
           scale_x_discrete(labels = c("Single", "Small", "Large")) +
           stat_pvalue_manual(stat.test, label = "p.adj.signif", size = 2) + # p.adjust
           #stat_compare_means(comparisons = my_comparisons, method = "wilcox.test", label = "p.format", p.adjust = 'fdr', size = 2, margin = margin(t = 0)) + 
           ggtitle(short_title)

  ggsave(complete_file_path, plot = plot1, device = "pdf", width = 2.5, height = 5)
}


In [ ]:
## Supplementary

In [ ]:
# Modules

### Figure S5A

### Figure S5B

In [ ]:
# in python
import os
import pandas as pd
from plotnine import ggplot, aes, geom_boxplot, geom_jitter, scale_color_manual, theme_minimal, theme, element_text, element_line, element_blank, labs, scale_x_discrete, xlab, ylab, ggtitle, geom_segment, annotate
from scipy.stats import ranksums
from statsmodels.stats.multitest import multipletests

def composition_analysis(compartment_seurat_object, category_meta, category_idents, out_dir, refined_label_meta, sampleID_meta, name):
    obj = compartment_seurat_object
    obs = obj.obs

    categories = {}

    for i in range(len(category_idents)):
        categories[f'cat{i+1}'] = obs[obs[category_meta] == category_idents[i]][sampleID_meta].unique()
    
    Dout = pd.crosstab(obs[refined_label_meta], obs[sampleID_meta], normalize='columns')
    subtypes = obs[refined_label_meta].unique()
    
    my_comparisons = [
        ('Young', 'Intermediate'),
        ('Young', 'Old'),
        ('Intermediate', 'Old')
    ]

    palette = {"Young": "#0091CA", "Intermediate": "#D3D3D3", "Old": "#D8423D"}
    
    for subtype in subtypes:
        print(f"Processing subtype: {subtype}")
        plot_data = []
        for i in range(len(category_idents)):
            cat_ids = categories[f'cat{i+1}']
            if not cat_ids.size:
                continue
            class_values = Dout.loc[subtype, cat_ids].values
            df = pd.DataFrame({
                'Status': category_idents[i],
                'Fraction': class_values,
                'ID': cat_ids,
                'Subtype': subtype
            })
            plot_data.append(df)
    
        if not plot_data:
            print(f"No data available for subtype: {subtype}")
            continue
    
        plot_df = pd.concat(plot_data, ignore_index=True)
        plot_df['Status'] = pd.Categorical(plot_df['Status'], categories=['Young', 'Intermediate', 'Old'], ordered=True)

        p = (
            ggplot(plot_df, aes(x='Status', y='Fraction', color='Status'))
            + geom_boxplot(size=0.5, outlier_shape="", show_legend=False, fatten=1.5)
            + geom_jitter(width=0.25, size=0.3, stroke=0.1)
            + scale_color_manual(values=palette)
            + theme_minimal()
            + theme(
                text=element_text(size=6, color="black"),
                axis_line=element_line(size=0.3, color="black"),
                axis_text_x=element_text(angle=0, ha="center", color="black", margin={'t': -5}),
                axis_text_y=element_text(angle=0, color="black", margin={'r': 4}),
                panel_grid_major=element_blank(),
                panel_grid_minor=element_blank(),
                legend_position="none"
            )
            + labs(y=subtype, x="")
            + scale_x_discrete(labels=["<40", "40-59", "≥60"])
            + xlab("age")
            + ylab("Proportion of CD8 T Cells")
            + ggtitle(subtype)
        )

        y_max = plot_df['Fraction'].max()
        offset = y_max * 0.15
        segment_offset = offset * 0.3

        p_values = []
        for group1, group2 in my_comparisons:
            group1_values = plot_df[plot_df['Status'] == group1]['Fraction']
            group2_values = plot_df[plot_df['Status'] == group2]['Fraction']
            if len(group1_values) > 0 and len(group2_values) > 0:
                stat, p_value = ranksums(group1_values, group2_values)
                p_values.append(p_value)
            else:
                p_values.append(1.0)
        
        reject, pvals_corrected, _, _ = multipletests(p_values, method='fdr_bh')

        for i, (group1, group2) in enumerate(my_comparisons):
            y_position = y_max + offset * (i + 1) - segment_offset
            x1 = category_idents.index(group1) + 1
            x2 = category_idents.index(group2) + 1
            p += geom_segment(aes(x=x1, xend=x1, y=y_position - offset / 2, yend=y_position - segment_offset), color='black', size=0.3)
            p += geom_segment(aes(x=x2, xend=x2, y=y_position - offset / 2, yend=y_position - segment_offset), color='black', size=0.3)
            p += geom_segment(aes(x=x1, xend=x2, y=y_position - segment_offset, yend=y_position - segment_offset), color='black', size=0.3)
            p += theme(
                        axis_title_x=element_text(margin={'t': 2}),
                        axis_title_y=element_text(margin={'r': 4})
            )
            p += annotate('text', x=(x1 + x2) / 2, y=y_position + offset / 10 - segment_offset, label=f'{pvals_corrected[i]:.2e}', ha='center', va='bottom', color='black', size=6)

        pdf_path = os.path.join(out_dir, f"{name}_{category_meta}_{subtype}_comp_boxplot.pdf")
        p.save(pdf_path, width=1, height=1.5, format='pdf')
        print(f"Plot for {subtype} saved to {pdf_path}")

########################
composition_analysis(data_filtered, 
                     'Age_group3', 
                     ['Young', 'Intermediate', 'Old'], 
                     '/sc/arion/work/kimg20/aging/comp/', 
                     'Cluster_names_all', 
                     'Tube_id', 
                     '240621_cd8_all')

### Figure S5C

### Figure S5D

### Figure S5E

### Figure S5F

In [ ]:
large_clones <- metadata_df %>%
  filter(cloneSize %in% c('Large (5 < X <= 500)'))

large_clones_tube_ids <- unique(large_clones$Tube_id)
large_clones_tube_ids
sce_filtered <- subset(sce, subset = Tube_id %in% large_clones_tube_ids)

In [ ]:
clonal_results <- clonalProportion(
  input.data = sce_filtered,
  cloneCall = "strict", 
  group.by = "Tube_id",
  chain = "both",
  clonalSplit = c(1, 100000),  # find top clone
  exportTable = TRUE  
)

In [ ]:
clonal_results_df <- as.data.frame(clonal_results, stringsAsFactors = FALSE)

clonal_results_df <- tibble::rownames_to_column(clonal_results_df, var = "Tube_id")
filtered_clonal_results <- clonal_results_df %>%
  filter(Tube_id %in% large_clones_tube_ids)

#print(filtered_clonal_results)

In [ ]:
total_cd8_counts <- sce_filtered@meta.data %>%
  group_by(Tube_id) %>%  
  summarize(total_cd8_count = n()) 

#print(total_cd8_counts)

In [ ]:
merged_results <- filtered_clonal_results %>%
  left_join(total_cd8_counts, by = "Tube_id") 

In [ ]:
merged_results1 <- merged_results1 %>%
  distinct(Tube_id, .keep_all = TRUE) 

In [ ]:
merged_results1 <- merged_results1 %>%
  mutate(proportion = `[1:1]` / total_cd8_count) 

#print(merged_results1)

In [ ]:
index_before_40 <- which(merged_results1$Tube_id == tube_id_before_40)
index_after_59 <- which(merged_results1$Tube_id == tube_id_after_59)

rows_before_40 <- merged_results1[(index_before_40-1):(index_before_40+1), ]
rows_after_59 <- merged_results1[(index_after_59-1):(index_after_59+1), ]

merged_rows <- rbind(rows_before_40, rows_after_59)
print(merged_rows)


In [ ]:
merged_results1 <- merged_results1 %>%
  arrange(Age) %>% 
  mutate(Tube_id = factor(Tube_id, levels = Tube_id)) 

tube_id_before_40 <- merged_results1 %>%
  filter(Age < 40) %>%
  tail(1) %>%
  pull(Tube_id)

tube_id_after_59 <- merged_results1 %>%
  filter(Age > 59) %>%
  head(1) %>%
  pull(Tube_id)

p <- ggplot(merged_results1, aes(x = Tube_id, y = proportion)) +
  geom_bar(stat = "identity") +
  labs(title = "Proportion of top clone in each sample out of CD8 T",
       x = "Tube ID",
       y = "Proportion") +
  theme_minimal() +
  theme(
    text = element_text(size = 6, colour = "black"),
    plot.title = element_text(size = 6, hjust = 0.5, margin = margin(t = -1.5)),
    axis.line = element_line(linewidth = 0.3, colour = "black"),
    axis.text.x = element_text(angle = 90, hjust = 0.5, vjust = 0.5, colour = "black", margin = margin(t = -1.5)),
    axis.text.y = element_text(angle = 0, colour = "black", margin = margin(r = -1)),
    axis.ticks = element_line(size = 0.25, colour = "black"),
    axis.ticks.length = unit(0.05, "cm"),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  geom_vline(xintercept = which(levels(merged_results1$Tube_id) == tube_id_before_40) + 0.5, linetype = "dashed", color = "red") +
  geom_vline(xintercept = which(levels(merged_results1$Tube_id) == tube_id_after_59) - 0.5, linetype = "dashed", color = "red") +
  geom_hline(yintercept = 0.05, linetype = "dashed", color = "black")

print(p)
ggsave("large_clones_proportion_plot.pdf", 
       plot = p, 
       device = "pdf", 
       width = 6.5, 
       height = 1)

### Figure S5G

In [ ]:
# Spearman Blood!!!!!!!!!!!!!!!!!!!!!
file <- "lollipops1014/cd8/"
metadata <- sce@meta.data


features <- c("Tm21", "Tm41", "Tm71", "Tm81", "Tm121", "exhaustion.markers1")

by_sample <- metadata %>%
  group_by(Tube_id, Age) %>%
  summarise(across(all_of(features), ~mean(.x, na.rm = TRUE)), .groups = 'drop')

# average IFNG expression
avg_expr <- AverageExpression(object = sce, features = "IFNG", assay = "RNA", group.by = c('Tube_id', 'Age'))$RNA
avg_expr <- as.data.frame(as.matrix(avg_expr))

avg_expr_long <- avg_expr %>%
  pivot_longer(cols = everything(), 
               names_to = "Tube_id_Age", 
               values_to = "IFNG") %>%
  separate(Tube_id_Age, into = c("Tube_id", "Age"), sep = "_")


by_sample <- by_sample %>%
  mutate(Age = as.numeric(Age))

avg_expr_long <- avg_expr_long %>%
  mutate(Age = as.numeric(Age))

by_sample <- by_sample %>%
  left_join(avg_expr_long, by = c("Tube_id", "Age"))


p_values <- data.frame(feature = features, p_value = NA, rho = NA)

for (feature in features) {
  test_result <- cor.test(by_sample[[feature]], by_sample$Age, method = "spearman", use = "complete.obs")
  
  p_values$p_value[p_values$feature == feature] <- test_result$p.value
  p_values$rho[p_values$feature == feature] <- test_result$estimate
}

p_values <- p_values %>%
  mutate(fdr_corrected_p_value = p.adjust(p_value, method = "fdr"),
         log_p_value = -log10(p_value),
         log_fdr_corrected_p_value = ifelse(rho > 0, 
                                            -log10(fdr_corrected_p_value), 
                                            log10(fdr_corrected_p_value)))

positive_threshold <- -log10(0.05) 
negative_threshold <- log10(0.05) 

p_values <- p_values %>%
  mutate(significant = log_fdr_corrected_p_value >= positive_threshold | log_fdr_corrected_p_value <= negative_threshold)
p_values <- p_values %>%
  mutate(color = ifelse(significant, 
                        ifelse(log_fdr_corrected_p_value > 0, "#D8423D", "#0091CA"), 
                        "#D3D3D3"))


plot <- ggplot(p_values, aes(x = reorder(feature, -log_fdr_corrected_p_value),  
                             y = log_fdr_corrected_p_value)) +
  geom_segment(aes(x = reorder(feature, -log_fdr_corrected_p_value), 
                   xend = reorder(feature, -log_fdr_corrected_p_value), 
                   y = 0, yend = log_fdr_corrected_p_value),
               color = "#D3D3D3") +
  geom_point(aes(color = color), size = 1) +
  scale_color_identity() +  
  geom_hline(yintercept = -log10(0.05), linetype = "dashed", color = "#D3D3D3") +
  geom_hline(yintercept = log10(0.05), linetype = "dashed", color = "#D3D3D3") +
  geom_hline(yintercept = log10(1),  color = "black") +

  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 0.5, size = 6),
    axis.ticks = element_line(color = "black"),  
    axis.ticks.length = unit(0.1, "cm"), 
    text = element_text(size = 6),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    panel.border = element_blank(),
    axis.line.y.left = element_line(size = 0.5),
    axis.line = element_line(size = 0.5)
  ) +
  labs(x = "Features", y = "-log10(p)") 


ggsave(filename = paste0(file, "blood_cd8_spearmann_mean_no5_percentclones.pdf"), plot = plot, width = 1, height = 1.5)
